In [1]:
# Imports

import os
import math
import random
import operator
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math, itertools
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import ShuffleSplit
from operator import itemgetter
from statistics import mean
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import Counter

from ipynb.fs.full.helper_fns import *

In [2]:
# Code that sets up values to construct all possible feature combinations.

poss = []

education_cmbs = [['tertiary', 'unknown'],['primary','secondary']]
marital_cmbs = [['single'],['married'],['divorced']]
job_cmbs = [['student','retired','unemployed'],['admin', 'management', 'self-employed'],['technician', 'unknown', 'services'],['housemaid', 'blue-collar', 'entrepreneur']]
default_cmbs = [['no'],['yes']]
loan_cmbs = [['no'],['yes']]
housing_cmbs = [['no'],['yes']]


poss.append(education_cmbs)
poss.append(marital_cmbs)
poss.append(job_cmbs)
poss.append(default_cmbs)
poss.append(loan_cmbs)
poss.append(housing_cmbs)
all_combs = list(itertools.product(*poss))

# Age query strings.
age_query_strings = ['age < 26','age >= 26 & age <=60','age >60']

# Balance query strings.
balance_query_strings = ['balance <= 5000',' balance > 5000']

# Max call number to consider.
# We obtained this by dividing the overall reward we have in the dataset by the number of successes.
max_calls = 20

In [3]:
def find_combinations(sub_attributes, ratios, att_name):
    num_iter = len(ratios)
    sil_scores = []
    # Making use of the K-Means algorithm ... number of centroids are from 2 to n-1.
    for clust_num in range(2, num_iter):
        kmeans = KMeans(n_clusters = clust_num)
        kmeans.fit(ratios.reshape(-1,1))
        results = kmeans.labels_
        sil_scores.append((silhouette_score(ratios.reshape(-1,1), results, metric='euclidean'), results, clust_num))
    # We make use of the silhouette score to determine the ideal number of centroids.
    sorted_sil_scores = sorted(sil_scores, key=lambda x: x[0], reverse = True)
    # We then use this ideal number of centroids to determine which sub attributes should be aggregated.
    joined_sub_attributes = []
    for i in range(0, sorted_sil_scores[0][2]):
        joined_sub_attributes.append([])
    join_list = sorted_sil_scores[0][1]
    for index, value in enumerate(join_list):
        pos = join_list[index]
        joined_sub_attributes[pos].append(sub_attributes[index])
    return_joined_sub_attributes = []
    for arr in joined_sub_attributes:
        query_str = ""
        for item in arr:
            single_str = att_name + " == " + item + " & "
            query_str += single_str
        query_str = query_str[:-2]
        return_joined_sub_attributes.append(query_str)
#     print(return_joined_sub_attributes)
    return return_joined_sub_attributes

# The following is the format of the way in which this method should be called.
# find_combinations(['a', 'b', 'c', 'd'], np.array([1, 4, 7, 90]), "job")

In [5]:
# Main code ... orchestrates everything!

# Pull and filter all calls <= 20.
current_dir = os.getcwd()
mkt_df = load_file(current_dir + '/bank-full.csv')
mkt_df_filtered = mkt_df[(mkt_df['campaign']>=1) & (mkt_df['campaign']<=max_calls)]

print(mkt_df_filtered.shape)


# Splitting dataframe into data and result dataframes.
X = mkt_df_filtered.iloc[:,0:len(mkt_df_filtered.columns)-1]
y = mkt_df_filtered.iloc[:,-1]   

kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    # Obtaining relevant dataframes based on the splitting defined by cross validation.
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    train = mkt_df_filtered.iloc[train_index]
    test = mkt_df_filtered.iloc[test_index]
    
    # At this point, we can run computations for the success rate of each sub attribute and join
    # the sub-attributes based on the output of k-means.
    
    
    # We can now go ahead and genreate the feature sets based on what was done previously.
    
    num_iter = 0
    num_non_zero_combs = 0
    combs_to_consider = {}
    # Setting up looping structures to generate all possibilities.
    # All that has to be done now is to change 'df_train' to 'X_train'.
    for age_query in age_query_strings:
        df_filtered_final = train.query(age_query)
        for bal_query in balance_query_strings:
            df_filtered_final = df_filtered_final.query(bal_query)
            for comb in all_combs:
                dict_final_query = construct_dict(comb)
                num_iter += 1
                extracted_df = extract_rows_feature_set(df_filtered_final, dict_final_query)
                key = (dict_final_query['education'], dict_final_query['job'], dict_final_query['marital'], dict_final_query['default'], dict_final_query['loan'], dict_final_query['housing'], bal_query, age_query)
                if extracted_df.shape[0] != 0:
                    num_non_zero_combs += 1
                    results = compute_expected_succ_per_call_rate_feature_set(extracted_df, max_calls)
                    max_loc = compute_optimal_call_no(results)
                    if max_loc >= 0:
                        combs_to_consider[key] = ('age':age_query, 'bal':bal_query, 'comb':comb, 'consider':True, 'max_loc':max_loc, 'rate':results[max_loc]['expected'])
                    else:
                        combs_to_consider[key] = ('age':age_query, 'bal':bal_query, 'comb':comb, 'consider':False)
                    print(age_query)
                    print(bal_query)
                    print(comb)
                    print(dict_final_query)
#                     print("Max Loc is: ", max_loc+1)
#                     plot_graph_new(results, max_calls)
                    print("\n\n\n")
#  When we are finished creating the feature combinations .... we can now use the hold out set for validation of the model!
    num_succ = 0
    num_calls = 0
    num_succ_optimal = 0
    num_calls_optimal = 0
    for loc, row in test.iterrows():
        # For regular method.
        if row['y'] == "yes":
            num_succ += 1
        num_calls += 1
        -----------------------------------------------------
        # For optimal method.
        # We have the exact values for each of the following:
        jb_query = find_matching_attribute_comb(str(row['job']), job_cmbs)
        mt_query = find_matching_attribute_comb(str(row['marital']), marital_cmbs)
        ec_query = find_matching_attribute_comb(str(row['education']), education_cmbs)
        house_query = find_matching_attribute_comb(str(row['housing']), housing_cmbs)
        loan_query = find_matching_attribute_comb(str(row['loan']), loan_cmbs)
        def_query = find_matching_attribute_comb(str(row['default']), default_cmbs)
        ##########################
        no_calls = row['campaign']
        # The balance and age are within ranges so we need to find the matching query.
        ##########################
        bal = row['balance']
        bal_query = None
        age = row['age']
        age_query = None
        for age_q in age_query_strings:
            if eval(age_q):
                age_query = age_q
        for bal_q in balance_query_strings:
            if eval(bal_q):
                bal_query = bal_q
        key_to_find = (ec_query, jb_query, mt_query, def_query, loan_query, house_query, bal_query, age_query)
        fs = combs_to_consider[key]
        if fs!= None:
            if fs['max_loc'] >= no_calls:
                if row['y'] == "yes":
                    num_succ_optimal += 1
                num_calls_optimal += no_calls
            else:
                num_calls_optimal += fs['max_loc']
        
        

(44967, 17)
age < 26
balance <= 5000
(['tertiary', 'unknown'], ['single'], ['student', 'retired', 'unemployed'], ['no'], ['no'], ['no'])
{'education': ['tertiary', 'unknown'], 'job': ['student', 'retired', 'unemployed'], 'marital': ['single'], 'default': ['no'], 'loan': ['no'], 'housing': ['no']}




age < 26
balance <= 5000
(['tertiary', 'unknown'], ['single'], ['student', 'retired', 'unemployed'], ['no'], ['no'], ['yes'])
{'education': ['tertiary', 'unknown'], 'job': ['student', 'retired', 'unemployed'], 'marital': ['single'], 'default': ['no'], 'loan': ['no'], 'housing': ['yes']}




age < 26
balance <= 5000
(['tertiary', 'unknown'], ['single'], ['student', 'retired', 'unemployed'], ['no'], ['yes'], ['no'])
{'education': ['tertiary', 'unknown'], 'job': ['student', 'retired', 'unemployed'], 'marital': ['single'], 'default': ['no'], 'loan': ['yes'], 'housing': ['no']}




age < 26
balance <= 5000
(['tertiary', 'unknown'], ['single'], ['student', 'retired', 'unemployed'], ['yes'], ['ye

age < 26
balance <= 5000
(['tertiary', 'unknown'], ['divorced'], ['technician', 'unknown', 'services'], ['no'], ['no'], ['yes'])
{'education': ['tertiary', 'unknown'], 'job': ['technician', 'unknown', 'services'], 'marital': ['divorced'], 'default': ['no'], 'loan': ['no'], 'housing': ['yes']}




age < 26
balance <= 5000
(['primary', 'secondary'], ['single'], ['student', 'retired', 'unemployed'], ['no'], ['no'], ['no'])
{'education': ['primary', 'secondary'], 'job': ['student', 'retired', 'unemployed'], 'marital': ['single'], 'default': ['no'], 'loan': ['no'], 'housing': ['no']}




age < 26
balance <= 5000
(['primary', 'secondary'], ['single'], ['student', 'retired', 'unemployed'], ['no'], ['no'], ['yes'])
{'education': ['primary', 'secondary'], 'job': ['student', 'retired', 'unemployed'], 'marital': ['single'], 'default': ['no'], 'loan': ['no'], 'housing': ['yes']}




age < 26
balance <= 5000
(['primary', 'secondary'], ['single'], ['student', 'retired', 'unemployed'], ['no'], ['yes'

age < 26
balance <= 5000
(['primary', 'secondary'], ['divorced'], ['technician', 'unknown', 'services'], ['no'], ['no'], ['no'])
{'education': ['primary', 'secondary'], 'job': ['technician', 'unknown', 'services'], 'marital': ['divorced'], 'default': ['no'], 'loan': ['no'], 'housing': ['no']}




age < 26
balance <= 5000
(['primary', 'secondary'], ['divorced'], ['technician', 'unknown', 'services'], ['no'], ['yes'], ['no'])
{'education': ['primary', 'secondary'], 'job': ['technician', 'unknown', 'services'], 'marital': ['divorced'], 'default': ['no'], 'loan': ['yes'], 'housing': ['no']}




age < 26
balance <= 5000
(['primary', 'secondary'], ['divorced'], ['housemaid', 'blue-collar', 'entrepreneur'], ['no'], ['no'], ['yes'])
{'education': ['primary', 'secondary'], 'job': ['housemaid', 'blue-collar', 'entrepreneur'], 'marital': ['divorced'], 'default': ['no'], 'loan': ['no'], 'housing': ['yes']}




age >= 26 & age <=60
balance <= 5000
(['tertiary', 'unknown'], ['single'], ['student', '

KeyboardInterrupt: 

In [29]:
age = 45
for age_q in age_query_strings:
    if eval(age_q):
        print("Yes!")
        print(age_q)

Yes!
age >= 26 & age <=60


In [53]:
def compute_metric(df):
    total_calls = 0
    total_successes = 0
    for loc, row in df.iterrows():
        if row['y'] == "yes":
            total_successes += 1
        total_calls += row['campaign']
    return div(total_successes, total_calls)

In [89]:
# Metric using our method.
filtered_combs_valid = [comb for comb in combs_to_consider if comb['consider'] == True]
filtered_combs_valid_sorted = sorted(filtered_combs_valid, key=lambda k: k['rate'], reverse = True) 
print("All combs: ", mean([comb['rate'] for comb in combs_to_consider]))
print("All combs(ratio >=0.20): ", mean([comb['rate'] for comb in combs_to_consider if comb['rate'] >= 0.20]))
# The ratio will be increased further when we filter the unwanted feature combinations.

All combs:  0.3513100131570743
All combs(ratio >=0.20):  0.5514576239792396


In [35]:
# Metric for the filtered dataset (only up to 20 calls).
print(compute_metric(mkt_df_filtered))

0.04469269018705815


In [36]:
# Metric for the entire dataset (up to 56 calls).
print(compute_metric(mkt_df))

0.042326899068472104


#### This section computes the metric for the purpose of determining the groupings within each feature (  ..... need to finish for the remainder of features).

In [57]:
def compute_metric_for_each_attribute(all_values, df, attrib):
    metric_vals = np.zeros(shape=(len(all_values),1))
    for index, value in enumerate(all_values):
        v_query = "{0} == '{1}'".format(attrib, value)
        metric_val = compute_metric(mkt_df_filtered.query(v_query))
        metric_vals[index] = metric_val
        print(v_query, metric_val)
    return metric_vals

In [40]:
# Age.
all_age_query_strings = ['age >= 10 & age <= 19', 'age >= 20 & age <= 29', 'age >= 30 & age <= 39', 'age >= 40 & age <= 49', 'age >= 50 & age <= 59','age >= 60 & age <= 69', 'age >= 70 & age <= 79', 'age >= 80']
for age_query in all_age_query_strings:
    df_filtered_final = mkt_df_filtered.query(age_query)
    print(age_query, compute_metric(df_filtered_final))

age >= 10 & age <= 19 0.14634146341463414
age >= 20 & age <= 29 0.07631601041054488
age >= 30 & age <= 39 0.040097307272879794
age >= 40 & age <= 49 0.03357253501090633
age >= 50 & age <= 59 0.03437390389337075
age >= 60 & age <= 69 0.12427647259107934
age >= 70 & age <= 79 0.20594965675057209
age >= 80 0.1950354609929078


In [77]:
# Occupation.
all_jobs = ['student', 'retired', 'unemployed', 'admin', 'management', 'self-employed', 'technician', 'unknown', 'services', 'housemaid', 'blue-collar', 'entrepreneur']
compute_metric_for_each_attribute(all_jobs, mkt_df_filtered, 'job')

job == 'student' 0.12907869481765835
job == 'retired' 0.09875598086124401
job == 'unemployed' 0.06631648063033486
job == 'admin' 0.0
job == 'management' 0.05079848502596541
job == 'self-employed' 0.04410377358490566
job == 'technician' 0.04040647274128299
job == 'unknown' 0.0379041248606466
job == 'services' 0.03470001880759827
job == 'housemaid' 0.0319243275199527
job == 'blue-collar' 0.02749087520385183
job == 'entrepreneur' 0.030182090296832127


In [59]:
# current_dir = os.getcwd()
# mkt_df = load_file(current_dir + '/bank-full.csv')
# mkt_df_filtered = mkt_df[(mkt_df['campaign']>=1) & (mkt_df['campaign']<=max_calls)]
# Marital Status
all_ms = ['married', 'single', 'divorced']
metric_vals = compute_metric_for_each_attribute(all_ms, mkt_df_filtered, 'marital')
find_combinations(all_ms, metric_vals, 'marital')

marital == 'married' 0.03762389773737097
marital == 'single' 0.05973928537934915
marital == 'divorced' 0.04741576459826193


['marital == married & marital == divorced ', 'marital == single ']

In [22]:
# Education
all_ed = ['tertiary', 'secondary', 'primary', 'unknown']
compute_metric_for_each_attribute(all_ed, mkt_df_filtered, 'education')

education == 'tertiary' 0.05604090002528161
education == 'secondary' 0.0410958904109589
education == 'primary' 0.03222282905516111
education == 'unknown' 0.05277486910994764


In [23]:
# Default
all_def = ['no', 'yes', 'unknown']
compute_metric_for_each_attribute(all_def, mkt_df_filtered, 'default')

default == 'no' 0.04515566754107414
default == 'yes' 0.021996615905245348
default == 'unknown' 0.0


In [24]:
# Housing
all_hs = ['no', 'yes', 'unknown']
compute_metric_for_each_attribute(all_hs, mkt_df_filtered, 'housing')

housing == 'no' 0.061309197293838
housing == 'yes' 0.030395519335452
housing == 'unknown' 0.0


In [25]:
# Loan
all_ln = ['no', 'yes', 'unknown']
compute_metric_for_each_attribute(all_ln, mkt_df_filtered, 'loan')

loan == 'no' 0.048405306237651706
loan == 'yes' 0.025356992860142796
loan == 'unknown' 0.0


In [29]:
# Balance
all_bal_query_strings = ['balance >= -100000 & balance <= -1', 'balance >= 0 & balance < 1000', 'balance >= 1000 & balance < 2000', 'balance >= 2000 & balance < 3000', 'balance >= 3000 & balance < 4000','balance >= 4000 & balance < 5000', 'balance >= 5000 & balance < 6000', 'balance >= 6000 & balance < 7000', 'balance >= 7000 & balance < 8000', 'balance >= 8000 & balance < 9000', 'balance >= 9000 & balance < 10000','balance >= 10000 & balance < 11000', 'balance >= 11000 & balance < 12000', 'balance >= 12000 & balance < 13000', 'balance >= 13000 & balance < 14000', 'balance >= 14000 & balance < 15000', 'balance >= 15000 & balance < 16000', 'balance >= 16000 & balance < 17000','balance >= 17000 & balance < 18000', 'balance >= 18000 & balance < 19000', 'balance >= 19000 & balance < 19000', 'balance >= 20000']
for bal_query in all_bal_query_strings:
    df_filtered_final = mkt_df_filtered.query(bal_query)
    print(bal_query, compute_metric(df_filtered_final))

balance >= -100000 & age <= -1 0.0
balance >= 0 & balance < 1000 0.03984136840916789
balance >= 1000 & balance < 2000 0.05351934051997464
balance >= 2000 & balance < 3000 0.0680968858131488
balance >= 3000 & balance < 4000 0.0689893862482695
balance >= 4000 & balance < 5000 0.06461086637298091
balance >= 5000 & balance < 6000 0.06534547402249598
balance >= 6000 & balance < 7000 0.04906542056074766
balance >= 7000 & balance < 8000 0.0671217292377702
balance >= 8000 & balance < 9000 0.052269601100412656
balance >= 9000 & balance < 10000 0.058823529411764705
balance >= 10000 & balance < 11000 0.09217877094972067
balance >= 11000 & balance < 12000 0.08365019011406843
balance >= 12000 & balance < 13000 0.09821428571428571
balance >= 13000 & balance < 14000 0.035897435897435895
balance >= 14000 & balance < 15000 0.058823529411764705
balance >= 15000 & balance < 16000 0.03418803418803419
balance >= 16000 & balance < 17000 0.0
balance >= 17000 & balance < 18000 0.028985507246376812
balance >= 